# EXAMPLE 5
## Explaining the experiment
In this example, we are investigating the configuration memory of an FPGA with 25,611,008 bits (of which aroun 128k are flip-flops, the rest SRAM cells) organized in 800.344x32 words. A golden file following the manufacturer's rules was uploaded on the FPGA and then irradiated in static mode in an only round. Later, the content was downloaded and compared with the initial file.  

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [1]:
##LOAD_PATH"# Optional"
push!(LOAD_PATH,"/home/francisco/VACACOVIDs/LELAPE-main/LELAPE/src")
#push!(LOAD_PATH, "PATH_TO_FOLDER_WITH_LELAPE.jl");
#if the following cell is not successfully executed, run this one with the details of your system.

In [2]:
using DelimitedFiles, LELAPE, BenchmarkTools

## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 32 bits
* Memory size in words: 800344.
* In FPGAs, it seems more likely to succeed the POS operation.
* Tests were static. No information about cycles is necessary.

Ok, let us use this information to set these variables:

In [3]:
LA = 955760 # Memory size in words
WordWidth = 32 # Selfexplaining.
Operation = "POS" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.

false

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN, CYCLE.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [82]:
DATA1 = readdlm("ExampleFPGA18.csv", ',', UInt32, '\n', skipstart=1)

87503×4 Matrix{UInt32}:
 0x00000081  0x00008000  0x00000000  0x00000001
 0x000000c7  0x08000000  0x00000000  0x00000001
 0x000000d1  0x20000000  0x00000000  0x00000001
 0x000000f5  0x00001000  0x00000000  0x00000001
 0x0000012c  0x04000000  0x00000000  0x00000001
 0x0000015c  0x00c00000  0x00000000  0x00000001
 0x0000016d  0x08000000  0x00000000  0x00000001
 0x000001c1  0x00400000  0x00000000  0x00000001
 0x000001d6  0x00000200  0x00000000  0x00000001
 0x000001db  0x40000000  0x00000000  0x00000001
 0x00000214  0x00080000  0x00000000  0x00000001
 0x00000220  0x00080000  0x00000000  0x00000001
 0x00000228  0x08000000  0x00000000  0x00000001
          ⋮                          
 0x000e9348  0x127e0000  0x00000000  0x00000001
 0x000e9357  0x00060000  0x00020000  0x00000001
 0x000e93ad  0x127e0000  0x00000000  0x00000001
 0x000e93bc  0x00060000  0x00020000  0x00000001
 0x000e9412  0x127e0000  0x00000000  0x00000001
 0x000e9421  0x00060000  0x00020000  0x00000001
 0x000e9477  0x127e0000  0

Good!! If you have correctly proceeded, a 124x4 unsigned integer matrix is loaded. Now, let us analyze the DATA1 but we need to define in advance several variables to set the analysis.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ function, which returns the MBUs present in DATA.Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [83]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA1[:,2], DATA1[:,3], WordWidth)

([1, 1, 1, 1, 1, 2, 1, 1, 1, 1  …  8, 1, 8, 1, 8, 1, 8, 1, 9, 1], Any[[15], [27], [29], [12], [26], [22, 23], [27], [22], [9], [30]  …  [17, 18, 19, 20, 21, 22, 25, 28], [18], [17, 18, 19, 20, 21, 22, 25, 28], [18], [17, 18, 19, 20, 21, 22, 25, 28], [18], [17, 18, 19, 20, 21, 22, 25, 28], [18], [17, 18, 19, 20, 21, 22, 24, 25, 28], [18]])

The following loop will show how many MBUs per number of flipped bits were observed:

In [84]:
TNMBUS = 0
for size = 1: WordWidth
    NMBUs = length(findall(MBUSize.==size))
    TNMBUS += NMBUs*size
    NMBUs!=0 ? println("$size-bit MBUs: ", NMBUs) : nothing
end
TNMBUS

1-bit MBUs: 20919
2-bit MBUs: 1308
3-bit MBUs: 215
4-bit MBUs: 55
5-bit MBUs: 15
6-bit MBUs: 1
7-bit MBUs: 4
8-bit MBUs: 215
9-bit MBUs: 279
10-bit MBUs: 1028
11-bit MBUs: 650
12-bit MBUs: 1112
13-bit MBUs: 2520
14-bit MBUs: 1036
15-bit MBUs: 312
16-bit MBUs: 365
17-bit MBUs: 189
18-bit MBUs: 220
19-bit MBUs: 592
20-bit MBUs: 1472
21-bit MBUs: 1352
22-bit MBUs: 1889
23-bit MBUs: 1955
24-bit MBUs: 2040
25-bit MBUs: 1875
26-bit MBUs: 871
27-bit MBUs: 434
28-bit MBUs: 623
29-bit MBUs: 146
30-bit MBUs: 174
31-bit MBUs: 1
32-bit MBUs: 43636


1833554

Interesting! There are some MBUs but... are they real or just the accumulation of SBUs. Let us perform an analysis of the presence of false 2- and 3-bit MBUs.


## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

In [36]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 500

500

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C1_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C1_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C1_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C1_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [57]:
C4_SCY, C4_MCU, C4_TRC, C4_SHF = DetectAnomalies_FullCheck(DATA1, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)
@btime C4_SCY, C4_MCU, C4_TRC, C4_SHF = DetectAnomalies_FullCheck(DATA1, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)


(UInt32[0x00000c9f 0x000000c0; 0x00000ca0 0x000000b7; 0x00001940 0x0000007e; 0x000025e0 0x0000007e], Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2))

Perhaps these matrices are hard to read since, for efficiency, they were returned in UInt32 format, even the number of occurrences!!! Execute the following instrucction for a better comprehension.

In [58]:
println("Elements appearing more than expected and passing the Self-Consistency test:\n")
for index in 1:length(C4_SCY[:, 1])
    println("Value: 0x", string(C4_SCY[index, 1], base=16, pad = 6), " --> ", Int(C4_SCY[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA1, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

Elements appearing more than expected and passing the Self-Consistency test:

Value: 0x000c9f --> 192.
Value: 0x000ca0 --> 183.
Value: 0x001940 --> 126.
Value: 0x0025e0 --> 126.

Only up to 6 repetitions are explained by randomness.

In this example, it is not worth to check the other sets since they did not yield any positive result. If you had had success, you would only have to do the following:

In [85]:
#C4_All = [C4_SCY; C4_MCU; C4_TRC; C4_SHF]

C4_All1 = convert(Matrix{UInt32}, [1 1
2 1
3230 1
3231 1
3232 1
3233 1
3234 1])

7×2 Matrix{UInt32}:
 0x00000001  0x00000001
 0x00000002  0x00000001
 0x00000c9e  0x00000001
 0x00000c9f  0x00000001
 0x00000ca0  0x00000001
 0x00000ca1  0x00000001
 0x00000ca2  0x00000001

## Grouping bitflips
Now, we have discovered those values relating pairs of pseudoaddresses. Now, let us go to group events in DATA. 

The first step consists in labeling all the pseudoaddresses and grouping their assigned indexes to a matrix containing information for the possible MCUs. It is an intermediate step and is done with the instruction _MCU_Indexes_ with the required and already defined parameters. 

In [87]:
#Labeled_addresses = MCU_Indexes(DATA1, Operation, C4_All[:, 1], UsePseudoAddress, WordWidth)
@time Labeled_addresses = MCU_Indexes(DATA1, Operation, C4_All[:, 1], UsePseudoAddress, WordWidth)

LoadError: InterruptException:

You should have got a 38x2 matrix so there are 38 2-bit MCUs.  

Now, we will classify addresses with _Classify_Addresses_in_MCU()_, using the previous matrix, _Labeled_addresses_. This function returns a vector of matrices. The first element is a matrix with pseudoaddresses related to 2-bit MCUs, and the second and last one to SBUs.

In [49]:
Events = Classify_Addresses_in_MCU(DATA1, Labeled_addresses, UsePseudoAddress, WordWidth)

128-element Vector{Any}:
 UInt32[0x018cf4a8 0x018d0148 … 0x01932b68 0x01933808; 0x018cf9c0 0x018d0660 … 0x01933080 0x01933d20; … ; 0x01cc6806 0x01cc74a6 … 0x01d29ec6 0x01d2ab66; 0x01cc6a84 0x01cc7724 … 0x01d2a144 0x01d2ade4]
 0×127 Matrix{UInt32}
 0×126 Matrix{UInt32}
 0×125 Matrix{UInt32}
 0×124 Matrix{UInt32}
 0×123 Matrix{UInt32}
 0×122 Matrix{UInt32}
 0×121 Matrix{UInt32}
 0×120 Matrix{UInt32}
 0×119 Matrix{UInt32}
 0×118 Matrix{UInt32}
 0×117 Matrix{UInt32}
 0×116 Matrix{UInt32}
 ⋮
 UInt32[0x0005332f 0x00053330 … 0x00053fd4 0x00053fd5; 0x004abfe6 0x004abfe7 … 0x004acc8a 0x004abfeb; 0x007d700c 0x007d700d … 0x007d7cb1 0x007d7cb2; 0x0082e064 0x0082e065 … 0x0082ed08 0x0082e069]
 UInt32[0x009e5daa 0x009e5dab … 0x009e6a4e 0x009e5daf; 0x010f53d6 0x010f53d7 … 0x010f607a 0x010f53db]
 UInt32[0x003e1b5f 0x003e1b60 … 0x003e2802 0x003e1b63; 0x00409588 0x0040958a … 0x0040a22c 0x0040958d; … ; 0x015e8a2c 0x015e8a2d … 0x015e96cf 0x015e8a30; 0x016290a5 0x016290a6 … 0x01629d48 0x016290a9]
 UInt32[0x

---------------------
Difficult to read, isn't it? The following instruction makes the content more readable:

In [50]:
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    if NMCUs != 0
        println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
        for row = 1:NMCUs
            for bit = 1:length(Events)-k+1
                print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
                bit != length(Events)-k+1 ? print(", ") : print("\n")

            end
        end
        println()
    end
end

Pseudoaddresses involved in 128-bit MCUs (6 events):
0x18cf4a8, 0x18d0148, 0x18d0de8, 0x18d1a88, 0x18d2728, 0x18d33c8, 0x18d4068, 0x18d4d08, 0x18d59a8, 0x18d6648, 0x18d72e8, 0x18d7f88, 0x18d8c28, 0x18d98c8, 0x18da568, 0x18db208, 0x18dbea8, 0x18dcb48, 0x18dd7e8, 0x18de488, 0x18df128, 0x18dfdc8, 0x18e0a68, 0x18e1708, 0x18e23a8, 0x18e3048, 0x18e3ce8, 0x18e4988, 0x18e5628, 0x18e62c8, 0x18e6f68, 0x18e7c08, 0x18e88a8, 0x18e9548, 0x18ea1e8, 0x18eae88, 0x18ebb28, 0x18ec7c8, 0x18ed468, 0x18ee108, 0x18eeda8, 0x18efa48, 0x18f06e8, 0x18f1388, 0x18f2028, 0x18f2cc8, 0x18f3968, 0x18f4608, 0x18f52a8, 0x18f5f48, 0x18f6be8, 0x18f7888, 0x18f8528, 0x18f91c8, 0x18f9e68, 0x18fab08, 0x18fb7a8, 0x18fc448, 0x18fd0e8, 0x18fdd88, 0x18fea28, 0x18ff6c8, 0x1900368, 0x1901008, 0x1901ca8, 0x1902948, 0x19035e8, 0x1904288, 0x1904f28, 0x1905bc8, 0x1906868, 0x1907508, 0x19081a8, 0x1908e48, 0x1909ae8, 0x190a788, 0x190b428, 0x190c0c8, 0x190cd68, 0x190da08, 0x190e6a8, 0x190f348, 0x190ffe8, 0x1910c88, 0x1911928, 0x19125c8, 0

0x469916, 0x469917, 0x469918, 0x46a5b6, 0x46a5b7, 0x46a5b8, 0x469919, 0x46a5b9, 0x46991a
0xaf8eb4, 0xaf8eb5, 0xaf8eb6, 0xaf9b54, 0xaf9b55, 0xaf9b56, 0xaf8eb7, 0xaf9b57, 0xaf9b58
0xb2b217, 0xb2b218, 0xb2b219, 0xb2beb7, 0xb2beb8, 0xb2beb9, 0xb2b21a, 0xb2beba, 0xb2bebb
0xe7f1b6, 0xe7f1b7, 0xe7f1b8, 0xe7fe56, 0xe7fe57, 0xe7fe58, 0xe7f1b9, 0xe7fe59, 0xe7f1ba
0x1465e2b, 0x1465e2d, 0x1466aca, 0x1466acb, 0x1466acc, 0x1466acd, 0x1465e2e, 0x1465e2f, 0x1466ace

Pseudoaddresses involved in 8-bit MCUs (30 events):
0x0dad4b, 0x0db9ec, 0x0dc68c, 0x0dc68d, 0x0dc68e, 0x0dd32d, 0x0dd32e, 0x0dd32f
0x102431, 0x102432, 0x102433, 0x1030d0, 0x1030d1, 0x1030d2, 0x1030d3, 0x102434
0x10dc2f, 0x10dc30, 0x10dc31, 0x10e8ce, 0x10e8cf, 0x10e8d0, 0x10e8d1, 0x10dc32
0x293756, 0x293757, 0x293758, 0x2943f5, 0x2943f6, 0x2943f7, 0x2943f8, 0x293759
0x33df3b, 0x33df3c, 0x33df3d, 0x33ebda, 0x33ebdb, 0x33ebdc, 0x33ebdd, 0x33df3e
0x469289, 0x46928a, 0x46928b, 0x469f28, 0x469f29, 0x469f2a, 0x469f2b, 0x46928c
0x4695db, 0x4695dc,

0xcec8c2, 0xcec8c3, 0xcec8c4, 0xced561, 0xced563
0xcfb399, 0xcfb39a, 0xcfc039, 0xcfc03a, 0xcfc03b
0xe2d6dd, 0xe2d6de, 0xe2d6df, 0xe2e37e, 0xe2e37f
0xe2d874, 0xe2d875, 0xe2e514, 0xe2e515, 0xe2e516
0xe463cb, 0xe463cc, 0xe463cd, 0xe4706b, 0xe4706c
0xe46a39, 0xe46a3a, 0xe476d7, 0xe476d8, 0xe476d9
0xe46c0f, 0xe46c10, 0xe478ae, 0xe478af, 0xe478b0
0xef0f9e, 0xef0f9f, 0xef1c3d, 0xef1c3e, 0xef1c40
0xef2e0b, 0xef2e0c, 0xef2e0d, 0xef3aab, 0xef3aac
0xf44506, 0xf44508, 0xf451a5, 0xf451a8, 0xf44509
0xf6248e, 0xf6248f, 0xf6312d, 0xf6312e, 0xf6312f
0xf7f267, 0xf7f268, 0xf7ff07, 0xf7ff08, 0xf7ff09
0xfc971c, 0xfc971d, 0xfca3ba, 0xfca3bb, 0xfca3bc
0x1053daf, 0x1053db0, 0x1053db1, 0x1054a50, 0x1054a51
0x10a7ac1, 0x10a7ac2, 0x10a8760, 0x10a8761, 0x10a8762
0x10bc12e, 0x10bc12f, 0x10bcdcd, 0x10bcdce, 0x10bcdcf
0x1179385, 0x1179386, 0x117a026, 0x117a027, 0x117a029
0x1201772, 0x1201773, 0x1201774, 0x1202412, 0x1202413
0x1220fee, 0x1220fef, 0x1221c8d, 0x1221c8e, 0x1221c8f
0x136e2fd, 0x136e2fe, 0x136ef9d, 0x136e

0xabd0b5, 0xabd0b6, 0xabdd54, 0xabdd55
0xaf5b44, 0xaf5b45, 0xaf67e3, 0xaf67e4
0xafed43, 0xafed44, 0xaff9e2, 0xaff9e3
0xb203c6, 0xb203c7, 0xb21065, 0xb21066
0xb27eaa, 0xb27eab, 0xb28b49, 0xb28b4a
0xb2b006, 0xb2b007, 0xb2bca7, 0xb2bca8
0xb3349e, 0xb3349f, 0xb3413d, 0xb3413e
0xb3ffa1, 0xb3ffa2, 0xb40c40, 0xb40c42
0xb40fe8, 0xb40fe9, 0xb41c87, 0xb41c88
0xb4427d, 0xb4427e, 0xb44f1c, 0xb44f1d
0xb4906f, 0xb49070, 0xb49d10, 0xb49d11
0xb73fb8, 0xb73fb9, 0xb74c57, 0xb74c58
0xb7d3cf, 0xb7d3d0, 0xb7e06e, 0xb7e06f
0xb7e717, 0xb7e718, 0xb7f3b6, 0xb7f3b7
0xb83c2f, 0xb848ce, 0xb85570, 0xb8620f
0xba99a8, 0xba99a9, 0xbaa647, 0xbaa648
0xbafac9, 0xbafaca, 0xbb0768, 0xbb0769
0xbba86a, 0xbba86b, 0xbbb509, 0xbbb50a
0xbcdaac, 0xbcdaad, 0xbce74b, 0xbce74c
0xbd3df2, 0xbd4a91, 0xbd5731, 0xbd63d0
0xbe8d9f, 0xbe8da0, 0xbe9a3e, 0xbe9a3f
0xbf50c2, 0xbf50c3, 0xbf5d61, 0xbf5d62
0xc05189, 0xc0518a, 0xc05e28, 0xc05e29
0xc22dca, 0xc22dcb, 0xc23a6b, 0xc23a6c
0xc3718d, 0xc3718f, 0xc37e2e, 0xc38ad0
0xc5868a, 0xc5868b, 0xc59

0x30de61, 0x30eb01, 0x30f7a0
0x327885, 0x328524, 0x3291c6
0x339d00, 0x33a9a2, 0x33b641
0x33dfd8, 0x33dfd9, 0x33ec78
0x3410ad, 0x3410ae, 0x341d4e
0x359ee3, 0x359ee5, 0x35ab83
0x35a7bf, 0x35b45e, 0x35b45f
0x35cf0e, 0x35dbae, 0x35dbaf
0x35f07d, 0x35fd1d, 0x35fd1e
0x36f115, 0x36fdb5, 0x370a54
0x37ce27, 0x37dac6, 0x37dac7
0x388010, 0x388caf, 0x388cb1
0x39146e, 0x39146f, 0x39210f
0x39340b, 0x39340e, 0x3940ad
0x394ce7, 0x395986, 0x395987
0x39da9e, 0x39da9f, 0x39e73e
0x3a9d78, 0x3aaa19, 0x3ab6b8
0x3aaa4a, 0x3aaa4b, 0x3ab6ea
0x3c5db3, 0x3c6a55, 0x3c76f5
0x3cb1a5, 0x3cb1a6, 0x3cb1a7
0x3db2d1, 0x3dbf70, 0x3dbf71
0x3e3cc6, 0x3e3cc8, 0x3e4965
0x3e7f1b, 0x3e7f1c, 0x3e8bbb
0x3f9ce7, 0x3f9ce8, 0x3fa987
0x40de8f, 0x40eb2e, 0x40eb2f
0x4115bc, 0x4115be, 0x41225e
0x41aab5, 0x41b754, 0x41b756
0x41b299, 0x41b29a, 0x41b29b
0x41c24d, 0x41c24e, 0x41ceed
0x425bd8, 0x426876, 0x426878
0x4407fc, 0x4407fd, 0x44149c
0x44363b, 0x4442da, 0x4442db
0x44b5be, 0x44b5bf, 0x44b5c0
0x453cdb, 0x453cdc, 0x45497b
0x454e8e, 0x45

...

0xf0dc53
0xf0dc57
0xf0de8a
0xf0e137
0xf0e730
0xf0f10f
0xf0f88d
0xf0fde9
0xf0fe5d
0xf0ffd7
0xf10523
0xf105a2
0xf107ff
0xf10f0f
0xf110e6
0xf11250
0xf11285
0xf11526
0xf1160b
0xf116cf
0xf11a63
0xf11b22
0xf12319
0xf128c5
0xf128c8
0xf1298f
0xf129b1
0xf1303f
0xf1354c
0xf13897
0xf13b27
0xf1405e
0xf146e8
0xf1517b
0xf1523b
0xf152b9
0xf15777
0xf15936
0xf15f6a
0xf16345
0xf16478
0xf17238
0xf17406
0xf17ff1
0xf18285
0xf182db
0xf183da
0xf18618
0xf19590
0xf1a544
0xf1a6d6
0xf1ad65
0xf1b670
0xf1b84d
0xf1b8f6
0xf1bfbb
0xf1c23f
0xf1c5a0
0xf1cd15
0xf1cfd6
0xf1d0ae
0xf1daa7
0xf1df22
0xf1e4e5
0xf1e7ca
0xf1e9ef
0xf1eca5
0xf1edd4
0xf1f9e6
0xf1fa32
0xf2041e
0xf211e3
0xf219b5
0xf2207b
0xf22b67
0xf23263
0xf235dc
0xf23770
0xf241fc
0xf249c3
0xf24a38
0xf24aa4
0xf24eab
0xf253f8
0xf254ff
0xf25731
0xf25854
0xf25ece
0xf2620d
0xf26ee1
0xf2715e
0xf2797b
0xf27acf
0xf27d56
0xf28097
0xf28437
0xf28d62
0xf29048
0xf298d2
0xf29a93
0xf29b09
0xf29d70
0xf29e35
0xf29f14
0xf29f71
0xf2a045
0xf2a09d
0xf2a0c5
0xf2a440
0xf2a648
0xf2a92a
0

0x104c404
0x104c7c0
0x104c9e0
0x104cd3d
0x104cdf5
0x104cf6a
0x104d110
0x104e2bf
0x104e454
0x104eee2
0x104f697
0x104fd8f
0x1050154
0x105027b
0x10502a0
0x10503e9
0x1050453
0x1050a82
0x1051050
0x1051b60
0x1051d1f
0x1052b5b
0x1052bd3
0x105300c
0x1053162
0x1053c42
0x1054045
0x105414b
0x105428c
0x10542ba
0x10543a7
0x10543e1
0x1054566
0x1054613
0x105465e
0x10546ad
0x1054709
0x10547e9
0x1054968
0x1054c1b
0x1054d18
0x1054d92
0x1055099
0x105511b
0x10551a8
0x1055298
0x10553d4
0x1055791
0x1055b0b
0x1055d9e
0x1055fe8
0x105601d
0x1056030
0x1056462
0x10564c1
0x105660c
0x10566ea
0x1056ad4
0x1057d63
0x1058845
0x1059177
0x105976a
0x10599a5
0x105a117
0x105a267
0x105a30b
0x105a558
0x105a60a
0x105aa74
0x105b7b8
0x105b885
0x105bee9
0x105c6f6
0x105ceb5
0x105e263
0x105e394
0x105e5cf
0x105ee3b
0x105f483
0x105f544
0x105f9a4
0x105fcd5
0x1060630
0x10608f4
0x1060a81
0x1061334
0x10613d3
0x106147a
0x106151d
0x1061863
0x1061ad8
0x106216b
0x1062829
0x1062b01
0x1062ce1
0x1062fe0
0x1063148
0x10636ca
0x106390d
0x1064032


0x118b023
0x118b356
0x118b45b
0x118b609
0x118bbd1
0x118c772
0x118cdff
0x118d1e5
0x118d29d
0x118d3b4
0x118d44c
0x118d710
0x118d7cb
0x118e1f5
0x118e275
0x118e9b1
0x118fa15
0x11904c9
0x1190c5f
0x11910d1
0x1191ba3
0x1191e25
0x1191f78
0x119209e
0x1192782
0x119279d
0x1192d37
0x11935d8
0x1193e70
0x1193ea3
0x11941e5
0x1194f18
0x1195277
0x1195498
0x11957cc
0x1195b53
0x1195d0b
0x1195d76
0x1195e3c
0x1195e59
0x1195ebf
0x1195fae
0x1196140
0x11961d5
0x1196258
0x1196450
0x1196608
0x1196749
0x1196b35
0x1196cd9
0x1196dfc
0x1196eda
0x1196f33
0x11973d0
0x11975dd
0x11976fd
0x1197cc8
0x1197fdc
0x1198003
0x1198580
0x11985b7
0x1198657
0x119884a
0x1198d85
0x1199009
0x119964c
0x1199987
0x1199e3d
0x119a8ab
0x119a935
0x119ab0b
0x119af36
0x119afe1
0x119b654
0x119b869
0x119c7de
0x119c915
0x119cdc4
0x119e115
0x119e403
0x119fdcf
0x11a02d0
0x11a0c94
0x11a1cbb
0x11a2419
0x11a2f4b
0x11a3302
0x11a41b4
0x11a41f8
0x11a4840
0x11a4c12
0x11a55ec
0x11a607d
0x11a672d
0x11a68dd
0x11a6f43
0x11a6fd0
0x11a7083
0x11a8400
0x11a8ad7


0x1356472
0x13570d8
0x1357781
0x1359640
0x1359750
0x1359924
0x1359944
0x1359a91
0x135a873
0x135a974
0x135b303
0x135bfd4
0x135c190
0x135c195
0x135cb1d
0x135ce4e
0x135d51f
0x135e2ce
0x135fd73
0x1360231
0x13608e5
0x1360a1c
0x13610e8
0x136160d
0x1364a31
0x1366430
0x1367e0b
0x1367e79
0x136877f
0x136890f
0x1368fb0
0x13695bf
0x1369f83
0x136a4f2
0x136abcf
0x136adb0
0x136bd36
0x136c763
0x136ded5
0x136df53
0x136e26b
0x136f7ff
0x1370544
0x137089f
0x1371665
0x13736a8
0x1373c30
0x1374f60
0x1375144
0x1376129
0x1377b1d
0x1379c12
0x1379dcb
0x137ab4f
0x137abc3
0x137acb1
0x137b5e4
0x137c501
0x137d08e
0x137e7a2
0x137f3d8
0x137f65b
0x137f6f5
0x138042c
0x1380540
0x1380561
0x138114b
0x13812de
0x1381eb4
0x13827e2
0x13833ce
0x13839ac
0x13841a6
0x13859d4
0x1385a3d
0x1385ab2
0x1385d21
0x1385f88
0x1386878
0x1386a03
0x13875d9
0x1389054
0x138bffb
0x138c216
0x138cd99
0x138ce0e
0x138d913
0x138f3f3
0x138ff32
0x138fffd
0x1390232
0x1390f8b
0x13914af
0x13915e5
0x13919e0
0x1391baa
0x1392430
0x1393076
0x1393486
0x13940af


0x15a6611
0x15a70cf
0x15a982d
0x15a9917
0x15aa14e
0x15aa2e0
0x15aadc0
0x15aafee
0x15ab024
0x15ad5cd
0x15ae0dc
0x15ae16b
0x15aee4a
0x15afbcf
0x15b09e3
0x15b13f0
0x15b1eb1
0x15b394a
0x15b5020
0x15b518a
0x15b5d79
0x15b662e
0x15b6668
0x15b6eb1
0x15b82c6
0x15b9328
0x15b940e
0x15b9f15
0x15babf3
0x15badf5
0x15bb1b7
0x15bb710
0x15bb719
0x15bb75f
0x15bb7bd
0x15bb94b
0x15bba59
0x15bbad4
0x15bc294
0x15bce7c
0x15bd194
0x15bd77d
0x15bf5e7
0x15bf95a
0x15c1c33
0x15c34d3
0x15c4c6f
0x15c4ddb
0x15c76d9
0x15c80ff
0x15c8372
0x15ca840
0x15caf7e
0x15cc338
0x15cc34f
0x15ccbe2
0x15cd739
0x15ce7b5
0x15ce885
0x15d0bc3
0x15d2845
0x15d2e81
0x15d3520
0x15d35b0
0x15d3b26
0x15d56ac
0x15d5850
0x15d5a98
0x15d5ba9
0x15d675f
0x15d6b7b
0x15d6cbf
0x15d72c3
0x15d7467
0x15d74a3
0x15d74b6
0x15d7b5c
0x15d7dd1
0x15d7ed0
0x15d7f8c
0x15d7fc1
0x15d80cc
0x15d815f
0x15d8162
0x15d898a
0x15d99c8
0x15db194
0x15db2b4
0x15db308
0x15dd985
0x15dd9c0
0x15de5c8
0x15df1c4
0x15e08f8
0x15e1324
0x15e1520
0x15e1533
0x15e21e6
0x15e2ffa
0x15e47b5


## Are there false 2-bit MCUs?
The only-SBU Model allows estimating the number of false 2-bit events. We can use an implemented function in LELAPE:

In [ ]:
NF2BIT = NF2BitMCUs(DATA4, LA, Operation, length(C4_All[:,1]), WordWidth, KeepCycles, UsePseudoAddress)
println("We expected $NF2BIT false 2-bit MCUs in this  experiment.")

## Analysis completed
You can find in this folder other actual data got from the same memory. Analyze them and, if you wish, add aditional cells combining the anomalous DV values and try to do the most accurate example.